In [35]:
import os
import pickle 
import numpy as np
import pandas as pd
import string

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import spacy

# from nltk import word_tokenize, sent_tokenize
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [36]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fatha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fatha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [37]:
spacy.prefer_gpu()

True

In [38]:
nlp = spacy.load("en_core_web_sm")

In [39]:
# column titles and label maps
columns = ['id','label','statement','subject','speaker','job title','state info','party','barely true','false','half-true','mostly-true','pants-on-fire','context']
label_map = {'pants-fire':-3, 'false':-2, 'barely-true':-1, 'half-true':1, 'mostly-true':2, 'true':3}

In [40]:
# load dataset files as pandas dataframes

train_data = pd.read_csv('train.tsv', sep='\t', header=None, names=columns)
val_data = pd.read_csv('valid.tsv', sep='\t', header=None, names=columns)
test_data = pd.read_csv('test.tsv', sep='\t', header=None, names=columns)

train_data.head()

,id,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


# Functions

## 7 March

In [15]:
# stop_words = set(stopwords.words('english'))
#
# def load_statement_vocab_dict(train_data):
#     vocabulary_dict = {}
#     if not os.path.exists('vocabulary.p'):
#         tokenizer = Tokenizer()
#         tokenizer.fit_on_texts(train_data['statement'])
#         vocabulary_dict = tokenizer.word_index
#         print(len(vocabulary_dict))
#         cp.dump(vocabulary_dict, open("vocabulary.p", "wb"))
#         print('Created Vocabulary Dictionary...')
#         print('Saved Vocabulary Dictionary...')
#     else:
#         print('Loading Vocabulary Dictionary...')
#         vocabulary_dict = cp.load(open("vocabulary.p", "rb"))
#     return vocabulary_dict
#
# def preprocess_statement(statement, vocabulary_dict):
#     statement = statement.lower() # lowercasing
#     doc = nlp(statement)
#     taglist = []
#     wordlist = []
#     for token in doc:
#         if not token.is_stop:
#             wordlist.append(token.lemma_)
#             taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())))
#     wordlist = ' '.join(wordlist)
#     text = text_to_word_sequence(wordlist)
#     val = [0] * 10
#     val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
#     return val, taglist



# def load_statement_vocab_dict(train_data):
#     vocabulary_dict = {}
#     if not os.path.exists('vocabulary.p'):
#         tokenizer = Tokenizer()
#         tokenizer.fit_on_texts(train_data['statement'])
#         vocabulary_dict = tokenizer.word_index
#         print(len(vocabulary_dict))
#         cp.dump(vocabulary_dict, open("vocabulary.p", "wb"))
#         print('Created Vocabulary Dictionary...')
#         print('Saved Vocabulary Dictionary...')
#     else:
#         print('Loading Vocabulary Dictionary...')
#         vocabulary_dict = cp.load(open("vocabulary.p", "rb"))
#     return vocabulary_dict

# def preprocess_statement(statement, vocabulary_dict):
#     statement = [w for w in statement.split(' ') if w not in stop_words]
#     statement = ' '.join(statement)
#     text = text_to_word_sequence(statement)
#     val = [0] * 10
#     val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
#     return val

In [16]:
# pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4,
#             'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9, 'X' : 9,
#             'PART' : 9, 'SYM' : 9, 'INTJ' : 9 }
#
# def get_pos(statement):
#     doc = nlp(statement)
#     taglist = []
#     for token in doc:
#         taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())))
#     return taglist

## Sunday 12 March

TO DO FOR the WEEK: <br>
    - add function to remove stopwords from pos_id X
1. og statement -> get pos -> pos_id
	- train_data['pos_id'] = train_data['statement'].apply(get_pos)

2. og statement -> # preprocess statement: words lemmatized and stopwords are removed -> pred_statement
	- train_data['pred_statement'] = train_data['statement'].apply(preprocess_statement)

3. pred_statement -> add to vocab dict: add preprocessed statements to vocab dict -> vocab_dict
	- vocabulary_dict = load_statement_vocab_dict(train_data)

4. vocab_dict, pred_statement -> getWordID(): turn preprocessed statements to vector -> val
	- train_data['word_id'] = train_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict)

**NOTE** it is generally fine to add pos tags without stopwords removed.
	in siddarthhari, padding used to put pos tags as separate input layer.

15 mar: <br>
    - extracted getWordId() from preprocess_statement()
    


In [7]:
pos_tags = {'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb', 
            'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction', 
            'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun', 
            'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun', 
            'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other', 
            'SCONJ': 'subord conjunction', 'SYM': 'symbol', 'VERB': 'verb'}

# pos_dict_1 = {'NOUN' : 1, 'VERB' : 2, 'ADP' : 3, 'PROPN' : 4, 'PUNCT' : 5, 
#             'DET' : 6, 'ADJ' : 7, 'NUM' : 8, 'ADV' : 9, 'PRON' : 10, 'X' : 11, 
#             'PART' : 11, 'SYM' : 11, 'INTJ' : 11 }

pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4, 
            'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9, 'X' : 9, 
            'PART' : 9, 'SYM' : 9, 'INTJ' : 9 }


# pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4, 
#             'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9, 'X' : 9, 
#             'PART' : 9, 'SYM' : 9, 'INTJ' : 9, 'DET' : 9 }  # DET tag as 9

def get_pos(statement):
    doc = nlp(statement)
    taglist = []
    deplist = []
    for token in doc:
        taglist.append(pos_dict.get(token.pos_,max(pos_dict.values())))
    return taglist



## 19 March
TODO: 
1. POS tags - incorporate stop words into pos tags 
- update: no need since separate input.
2. preprocess_statement() - configure best ones. confirm lemmatizer works

In [50]:
pos_tags = {'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb', 
            'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction', 
            'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun', 
            'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun', 
            'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other', 
            'SCONJ': 'subordinating conjunction', 'SYM': 'symbol', 'VERB': 'verb', 'STOP': 'stop word'}

pos_dict = {'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CONJ': 4,
            'DET': 5, 'INTJ': 6, 'NOUN': 7, 'NUM': 8, 'PART': 9,
            'PRON': 10, 'PROPN': 11, 'PUNCT': 12, 'X': 13,
            'SCONJ': 14, 'SYM': 15, 'VERB': 16, 'STOP': 17}

def get_pos(statement):
    doc = nlp(statement)
    taglist = []
    for token in doc:
        if token.is_stop:
            taglist.append(17)
        else:
            taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())+1))
    return taglist

train_data['pos_id'] = train_data['statement'].apply(get_pos)
val_data['pos_id'] = val_data['statement'].apply(get_pos)
test_data['pos_id'] = test_data['statement'].apply(get_pos)

In [42]:
stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

# remove punct, stopwords, and lemmatize
def preprocess_statement(statement):
    statement = statement.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    statement = ''.join([i for i in statement if not i.isdigit()]) # remove numbers
    # statement = [w for w in statement.split(' ') if w not in stopwords.words('english')] # only stopwords
    statement = [lemmatizer.lemmatize(w) for w in statement.split(' ') if w not in stopwords.words('english')]
    statement = ' '.join(statement)
    return statement


In [43]:
def load_statement_vocab_dict(train_data):
    vocabulary_dict = {}
    if not os.path.exists('vocabulary.p'):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(train_data['pred_statement'])
        vocabulary_dict = tokenizer.word_index
        print(len(vocabulary_dict))
        with open("vocabulary.p", "wb") as f:
            pickle.dump(vocabulary_dict, f)
        print('Created Vocabulary Dictionary...')
        print('Saved Vocabulary Dictionary...')
    else:
        print('Loading Vocabulary Dictionary...')
        with open("vocabulary.p", "rb") as f:
            vocabulary_dict = pickle.load(f)
    return vocabulary_dict

In [44]:
# preprocessed statement to vector 
def getWordId(pred_statement, vocabulary_dict):
    text = text_to_word_sequence(pred_statement)
    val = [0] * 10
    val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
    return val

# Preprocessing

## DataFrame Cleaning

- train_data['label'] -> num values
- drops all rows where 'subject' and 'speaker' columns are missing
- drops all rows that have missing values in 'statement'
- drops context column

<h3>Mar 19 todo:</h3>

- refactor to single function
- instead of dropping rows, replace missing values in 'subject' and 'speaker' as empty string

In [45]:
# maps labels to numerical values
train_data['label'] = train_data['label'].map(label_map) 

# drop rows where 'subject' and 'speaker' are missing
train_data.drop(index=train_data[train_data.subject.isna()].index, inplace=True) 
train_data.drop(index=train_data[train_data.speaker.isna()].index, inplace=True)

# drop rows where there are missing values in 'statement'
train_data.drop(index=train_data[train_data.statement==' '].index, inplace=True)
train_data.drop(index=train_data[train_data.statement=='  '].index, inplace=True)
train_data.drop(index=train_data[train_data.statement=='\n'].index, inplace=True)

# drop context column 
train_data.drop(columns=['context'],inplace=True)

train_data = train_data.reset_index()
print('train count')
train_data.count()

train count


index            10238
id               10238
label            10238
statement        10238
subject          10238
speaker          10238
job title         7343
state info        8032
party            10238
barely true      10238
false            10238
half-true        10238
mostly-true      10238
pants-on-fire    10238
dtype: int64

In [46]:
# loads valid.tsv as pandas dataframe
# valid_data = pd.read_csv('valid.tsv',sep='\t',header=None, names=columns)

# maps labels to numerical values
val_data['label'] = val_data['label'].map(label_map) 

# drop rows where 'subject' and 'speaker' are missing
val_data.drop(index=val_data[val_data.subject.isna()].index, inplace=True)
val_data.drop(index=val_data[val_data.speaker.isna()].index, inplace=True)

# drop rows where there are missing values in 'statement'
val_data.drop(index=val_data[val_data.statement==' '].index, inplace=True)
val_data.drop(index=val_data[val_data.statement=='  '].index, inplace=True)
val_data.drop(index=val_data[val_data.statement=='\n'].index, inplace=True)

# drop context column 
val_data.drop(columns=['context'],inplace=True)

val_data = val_data.reset_index()
print('valid count')
val_data.count()

valid count


index            1284
id               1284
label            1284
statement        1284
subject          1284
speaker          1284
job title         939
state info       1005
party            1284
barely true      1284
false            1284
half-true        1284
mostly-true      1284
pants-on-fire    1284
dtype: int64

In [47]:
# loads test.tsv as pandas dataframe
# test_data = pd.read_csv('test.tsv',sep='\t',header=None, names=columns)
## print(len(test))

# maps labels to numerical values
test_data['label'] = test_data['label'].map(label_map)
## test.dropna(inplace=True)

# drop rows where 'subject' and 'speaker' are missing
test_data.drop(index=test_data[test_data.subject.isna()].index, inplace=True)
test_data.drop(index=test_data[test_data.speaker.isna()].index, inplace=True)

# drop rows where there are missing values in 'statement'
test_data.drop(index=test_data[test_data.statement==' '].index, inplace=True)
test_data.drop(index=test_data[test_data.statement=='  '].index, inplace=True)
test_data.drop(index=test_data[test_data.statement=='\n'].index, inplace=True)

# drop context column 
test_data.drop(columns=['context'],inplace=True)
print('test count')
test_data.count()

test count


id               1267
label            1267
statement        1267
subject          1267
speaker          1267
job title         942
state info       1005
party            1267
barely true      1267
false            1267
half-true        1267
mostly-true      1267
pants-on-fire    1267
dtype: int64

In [48]:
print('real news count')
train_data[train_data['label']>0].count()

real news count


index            5752
id               5752
label            5752
statement        5752
subject          5752
speaker          5752
job title        4264
state info       4663
party            5752
barely true      5752
false            5752
half-true        5752
mostly-true      5752
pants-on-fire    5752
dtype: int64

## ['statement']  

### get pos tag

In [51]:
train_data['pos_id'] = train_data['statement'].apply(get_pos)
val_data['pos_id'] = val_data['statement'].apply(get_pos)
test_data['pos_id'] = test_data['statement'].apply(get_pos)

In [52]:
train_data.head()

,index,id,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,pos_id
0,0,2635.json,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,"[16, 17, 11, 11, 0, 7, 16, 17, 12, 7, 7, 17, 7..."
1,1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,"[17, 17, 17, 7, 17, 7, 7, 12, 17, 16, 17, 0, 7..."
2,2,324.json,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"[11, 11, 16, 17, 11, 11, 12, 17, 16, 17, 17, 1..."
3,3,1123.json,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,"[7, 7, 7, 7, 17, 0, 17, 16, 0, 7, 7, 7, 12]"
4,4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,"[17, 0, 7, 16, 17, 17, 7, 17, 17, 7, 12]"


In [53]:
val_data.head()

,index,id,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,pos_id
0,0,12134.json,-1,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,"[17, 17, 17, 11, 16, 17, 17, 17, 17, 7, 12]"
1,1,238.json,-3,"When Obama was sworn into office, he DID NOT u...","obama-birth-certificate,religion",chain-email,NaN,NaN,none,11,43,8,5,105,"[17, 11, 17, 16, 17, 7, 12, 17, 17, 17, 16, 17..."
2,2,7891.json,-2,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,"[16, 16, 7, 16, 17, 17, 0, 7, 7, 17, 17, 17, 1..."
3,3,8169.json,1,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,"[16, 2, 7, 17, 11, 7, 17, 0, 12]"
4,4,929.json,1,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,"[17, 7, 17, 11, 17, 17, 7, 17, 17, 0, 7, 7, 17..."


### preprocess statements
TODO 19 mar:
- configure preprocessing/lemmatizing

In [54]:
train_data['pred_statement'] = train_data['statement'].apply(preprocess_statement)
# train_data.head()

In [55]:
vocabulary_dict = load_statement_vocab_dict(train_data)

Loading Vocabulary Dictionary...


In [56]:
train_data['word_id'] = train_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict))
# train_data.drop('pred_statement', axis=1, inplace=True)

In [57]:
train_data.head()

,index,id,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,pos_id,pred_statement,word_id
0,0,2635.json,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,"[16, 17, 11, 11, 0, 7, 16, 17, 12, 7, 7, 17, 7...",Says Annies List political group support third...,"[1, 6127, 880, 423, 324, 66, 4432, 136, 1376]"
1,1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,"[17, 17, 17, 7, 17, 7, 7, 12, 17, 16, 17, 0, 7...",When decline coal start It started natural gas...,"[107, 1687, 917, 762, 200, 497, 959, 217, 117,..."
2,2,324.json,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"[11, 11, 16, 17, 11, 11, 12, 17, 16, 17, 17, 1...",Hillary Clinton agrees John McCain voting give...,"[73, 47, 3054, 129, 190, 245, 192, 220, 115, 2..."
3,3,1123.json,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,"[7, 7, 7, 7, 17, 0, 17, 16, 0, 7, 7, 7, 12]",Health care reform legislation likely mandate ...,"[12, 14, 172, 211, 541, 409, 354, 542, 218, 3056]"
4,4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,"[17, 0, 7, 16, 17, 17, 7, 17, 17, 7, 12]",The economic turnaround started end term,"[2, 236, 3586, 497, 284, 237]"


### compare preprocessed statements with original 

In [58]:
print(train_data['statement']) 
print(train_data['pred_statement'])

0        Says the Annies List political group supports ...
1        When did the decline of coal start? It started...
2        Hillary Clinton agrees with John McCain "by vo...
3        Health care reform legislation is likely to ma...
4        The economic turnaround started at the end of ...
                               ...                        
10233    There are a larger number of shark attacks in ...
10234    Democrats have now become the party of the [At...
10235    Says an alternative to Social Security that op...
10236    On lifting the U.S. Cuban embargo and allowing...
10237    The Department of Veterans Affairs has a manua...
Name: statement, Length: 10238, dtype: object
0        Says Annies List political group support third...
1        When decline coal start It started natural gas...
2        Hillary Clinton agrees John McCain voting give...
3        Health care reform legislation likely mandate ...
4                 The economic turnaround started end term
          

In [59]:
print(train_data.loc[25,'statement'])
print(train_data.loc[25,'pred_statement'])

I dont know who (Jonathan Gruber) is.
I dont know Jonathan Gruber


In [61]:
# print(train_data['pred_statement']) 
ROW = 4213
print(f"original statements:\n{train_data.loc[ROW, 'statement']}")
print(f"pos tag ids:\n{train_data.loc[ROW, 'pos_id']}") 
print(f"preprocessed statements:\n{train_data.loc[ROW, 'pred_statement']}")
print(f"pred_statements vectorized:\n{train_data.loc[ROW, 'word_id']}")

original statements:
We worked together to enact ... the largest tax cut in Florida history.
pos tag ids:
[17, 16, 17, 17, 16, 12, 17, 0, 7, 7, 17, 11, 7, 12]
preprocessed statements:
We worked together enact  largest tax cut Florida history
pred_statements vectorized:
[33, 557, 891, 3865, 227, 6, 24, 62, 130]


In [27]:
# # print(train_data['pred_statement']) 
# ROW = 0
# print(train_data.loc[ROW,'statement'])
# print(train_data.loc[ROW,'pos_id'])
# print(train_data.loc[ROW,'pred_statement'])
# print(train_data.loc[ROW,'word_id'])

Says the Annies List political group supports third-trimester abortions on demand.
pos tag ids:
[1, 10, 3, 3, 6, 0, 1, 10, 4, 0, 0, 10, 0, 4]
Says Annies List political group support thirdtrimester abortion demand
[1, 6127, 880, 423, 324, 66, 4432, 136, 1376]


# statement 8 mar 

In [23]:
vocabulary_dict = load_statement_vocab_dict(train_data)

12404
Created Vocabulary Dictionary...
Saved Vocabulary Dictionary...


In [ ]:
train_data['word_id'] = train_data['statement'].apply(preprocess_statement)

In [24]:
train_data[['word_id', 'pos_id']] = train_data['statement'].apply(lambda x: pd.Series(preprocess_statement(x, vocabulary_dict)))

In [25]:
train_data.head()

,index,id,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,word_id,pos_id
0,0,2635.json,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,"[302, 1140, 520, 620, 207, 5118, 269, 1597]","[1, 0, 1, 6, 0, 1, 4, 0, 0, 0, 4]"
1,1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,"[2090, 964, 866, 866, 1141, 314, 249, 866, 195...","[0, 0, 0, 4, 1, 6, 0, 1, 1, 1, 4, 3, 3, 3, 4, ..."
2,2,324.json,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"[127, 101, 1658, 190, 254, 168, 310, 166, 1093...","[3, 3, 1, 3, 3, 4, 1, 3, 3, 0, 0, 3, 4, 4]"
3,3,1123.json,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,"[32, 43, 265, 298, 666, 667, 402, 467, 417, 11...","[0, 0, 0, 0, 6, 1, 6, 0, 0, 0, 4]"
4,4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,"[329, 4148, 866, 407, 505]","[6, 0, 1, 0, 0, 4]"


In [13]:
train_data.head()

,index,id,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,word_id
0,0,2635.json,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,"([302, 1140, 520, 620, 207, 5118, 269, 1597], ..."
1,1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,"([2090, 964, 866, 866, 1141, 314, 249, 866, 19..."
2,2,324.json,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"([127, 101, 1658, 190, 254, 168, 310, 166, 109..."
3,3,1123.json,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,"([32, 43, 265, 298, 666, 667, 402, 467, 417, 1..."
4,4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,"([329, 4148, 866, 407, 505], [6, 0, 1, 0, 0, 4])"


In [40]:
# stop_words = set(stopwords.words('english'))

# def load_statement_vocab_dict(train_data):
#     vocabulary_dict = {}
#     if not os.path.exists('vocabulary.p'):
#         tokenizer = Tokenizer()
#         tokenizer.fit_on_texts(train_data['statement'])
#         vocabulary_dict = tokenizer.word_index
#         print(len(vocabulary_dict))
#         cp.dump(vocabulary_dict, open("vocabulary.p", "wb"))
#         print('Created Vocabulary Dictionary...')
#         print('Saved Vocabulary Dictionary...')
#     else:
#         print('Loading Vocabulary Dictionary...')
#         vocabulary_dict = cp.load(open("vocabulary.p", "rb"))
#     return vocabulary_dict



### pre pos tag

# def preprocess_statement(statement, vocabulary_dict):
#     statement = statement.lower() # lowercasing
#     statement = [w for w in statement.split(' ') if w not in stop_words] # stop word removal
#     statement = [lemmatizer.lemmatize(w) for w in statement]  # lemmatization 
#     statement = ' '.join(statement)
    
# #   maps each word in the preprocessed statement to an integer index value in the vocabulary_dict, allowing us to represent each statement as a sequence of integers that can be used as input to a machine learning model.
#     text = text_to_word_sequence(statement)
#     val = [0] * 10
#     val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
#     return val

### doest work

# def preprocess_statement(statement, vocabulary_dict):
#     statement = statement.lower() # lowercasing
#     statement = [lemmatizer.lemmatize(w) for w in statement.split(' ')]  # lemmatization 
#     pos_tags = get_pos(' '.join(statement))
#     statement = [w for i, w in enumerate(statement) if pos_dict.get(pos_tags[i], 0) not in [2, 4] and w not in stop_words] # stop word removal after pos tagging
#     statement = ' '.join(statement)
    
#     # maps each word in the preprocessed statement to an integer index value in the vocabulary_dict, allowing us to represent each statement as a sequence of integers that can be used as input to a machine learning model.
#     text = text_to_word_sequence(statement)
#     val = [0] * 10
#     val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
#     return val

In [48]:
# def preprocess_statement(statement, vocabulary_dict):
#     statement = statement.lower() # lowercasing
#     doc = nlp(statement)
#     taglist = []
#     wordlist = []
#     for token in doc:
#         if not token.is_stop:
#             wordlist.append(token.lemma_)
#             taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())))
#     wordlist = ' '.join(wordlist)
#     text = text_to_word_sequence(wordlist)
#     val = [0] * 10
#     val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
#     return val, taglist

# vocabulary_dict = load_statement_vocab_dict(train_data)

# train_data[['word_id', 'pos_id']] = train_data['statement'].apply(lambda x: pd.Series(preprocess_statement(x, vocabulary_dict)))
# valid_data[['word_id', 'pos_id']] = valid_data['statement'].apply(lambda x: pd.Series(preprocess_statement(x, vocabulary_dict)))
# test_data[['word_id', 'pos_id']] = test_data['statement'].apply(lambda x: pd.Series(preprocess_statement(x, vocabulary_dict)))

Loading Vocabulary Dictionary...


In [ ]:
pos_tags = {'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb', 
            'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction', 
            'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun', 
            'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun', 
            'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other', 
            'SCONJ': 'subord conjunction', 'SYM': 'symbol', 'VERB': 'verb'}

pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4, 
            'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9, 'X' : 9, 
            'PART' : 9, 'SYM' : 9, 'INTJ' : 9 }

dep_dict = {'ACL' : 0, 'ACOMP' : 1, 'ADVCL' : 2, 'ADVMOD' : 3, 'AGENT' : 4, 
            'AMOD' : 5, 'APPOS' : 6, 'ATTR' : 7, 'AUX' : 8, 'AUXPASS' : 9, 
            'CASE' : 10, 'CC' : 11, 'CCOMP' : 12, 'COMPOUND' : 13, 'CONJ' : 14, 
            'CSUBJ' : 15, 'CSUBJPASS' : 16, 'DATIVE' : 17, 'DEP' : 18, 
            'DET' : 19, 'DOBJ' : 20, 'EXPL' : 21, 'INTJ' : 22, 'MARK' : 23, 
            'META' : 24, 'NEG' : 25, 'NOUNMOD' : 26, 'NPMOD' : 27, 'NSUBJ' : 28, 
            'NSUBJPASS' : 29, 'NUMMOD' : 30, 'OPRD' : 31, 'PARATAXIS' : 32, 
            'PCOMP' : 33, 'POBJ' : 34, 'POSS' : 35, 'PRECONJ' : 36, 'PREDET' : 37, 
            'PREP' : 38, 'PRT' : 39, 'PUNCT' : 40, 'QUANTMOD' : 41, 
            'RELCL' : 42, 'ROOT' : 43, 'XCOMP' : 44}

def get_pos(statement):
    doc = nlp(statement)
    taglist = []
    deplist = []
    for token in doc:
        taglist.append(pos_dict.get(token.pos_,max(pos_dict.values())))
        #deplist.append(token.dep_)
    return taglist

train_data['pos_id'] = train_data['statement'].apply(get_pos)
val_data['pos_id'] = val_data['statement'].apply(get_pos)
test_data['pos_id'] = test_data['statement'].apply(get_pos)


In [46]:
vocabulary_dict = load_statement_vocab_dict(train_data)

# train_data['word_id'] = train_data['statement'].apply(lambda x: preprocess_statement(x, vocabulary_dict))

valid_data['word_id'] = valid_data['statement'].apply(lambda x: preprocess_statement(x, vocabulary_dict))
test_data['word_id'] = test_data['statement'].apply(lambda x: preprocess_statement(x, vocabulary_dict))

Loading Vocabulary Dictionary...


In [47]:
pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4, 
            'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9, 'X' : 9, 
            'PART' : 9, 'SYM' : 9, 'INTJ' : 9 }

def get_pos(statement):
    doc = nlp(statement)
    taglist = []
    for token in doc:
        taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())))
    return taglist

In [43]:
train_data['pos_id'] = train_data['statement'].apply(get_pos)
val_data['pos_id'] = val_data['statement'].apply(get_pos)
test_data['pos_id'] = test_data['statement'].apply(get_pos)

In [49]:
print(train_data.head())

   index          id  label  \
0      0   2635.json     -2   
1      1  10540.json      1   
2      2    324.json      2   
3      3   1123.json     -2   
4      4   9028.json      1   

                                           statement  \
0  Says the Annies List political group supports ...   
1  When did the decline of coal start? It started...   
2  Hillary Clinton agrees with John McCain "by vo...   
3  Health care reform legislation is likely to ma...   
4  The economic turnaround started at the end of ...   

                              subject         speaker             job title  \
0                            abortion    dwayne-bohac  State representative   
1  energy,history,job-accomplishments  scott-surovell        State delegate   
2                      foreign-policy    barack-obama             President   
3                         health-care    blog-posting                   NaN   
4                        economy,jobs   charlie-crist                   NaN   

 

In [15]:
vocabulary_dict = load_statement_vocab_dict(train_data)

12404
Created Vocabulary Dictionary...
Saved Vocabulary Dictionary...


In [16]:
train_data['pos_id'] = train_data['statement'].apply(get_pos)
# val_data['pos_id'] = val_data['statement'].apply(get_pos)
# test_data['pos_id'] = test_data['statement'].apply(get_pos)

In [ ]:
def add(a,b):
